In [13]:
from urllib import request
my_dict = { 
    1:22,
    2:24,
    3:23,
    4:25,
    5:3,
    6:4,
    7:8,
    8:19,
    9:20,
    10:21,
    11:9,
    13:10,
    14:11,
    15:12,
    16:13,
    17:14,
    18:15,
    19:16,
    21:17,
    22:18,
    23:6,
    24:1,
    25:2,
    26:7,
    27:5 
   }
def preprocess_raw_data(line): 
    if '/' in line:
        return ''         
    line = line.replace(' ', ',',2)   
    return (line + '\n') #след. строка будет с новой строки

def loadVHI(my_dict):
    url = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_provinceData.php?country=UKR&provinceID={}&year1=1981&year2=2019&type=Mean"
    for i in my_dict:
      #  for i in range(1, 28): 
      #  if i == 12 or i == 20: # Киев,Севастополь
          #  continue
        j = swap_id(i,my_dict) 
        print('{}{}{}'.format('loading vhi_', j, '.csv'))   #выводим на экран файл, который сейчас загружается
        local_url = url.format(str(i)) #переводим в формат строки инт и пихаем в адресс
        response = request.urlopen(local_url)#открываем данные для скачивания 
        csv = response.read() 
        csv_str = str(csv)  
        lines = csv_str.split("\\n") 
        with open("vhi_" + str(j) + ".csv", "w") as fx:
            fx.write("year,week,SMN,SMT,VCI,TCI,VHI\n") 
            for line in lines:   # записываем все строки в наш файл
                fx.write(preprocess_raw_data(line)) #c помощью этой функции записываем в файл исправленные данные
                

def swap_id(i,my_dict): # меняем местами ай ди 

    return my_dict[i]#выводим 

In [15]:
loadVHI(my_dict)

loading vhi_22.csv
loading vhi_24.csv
loading vhi_23.csv
loading vhi_25.csv
loading vhi_3.csv
loading vhi_4.csv
loading vhi_8.csv
loading vhi_19.csv
loading vhi_20.csv
loading vhi_21.csv
loading vhi_9.csv
loading vhi_10.csv
loading vhi_11.csv
loading vhi_12.csv
loading vhi_13.csv
loading vhi_14.csv
loading vhi_15.csv
loading vhi_16.csv
loading vhi_17.csv
loading vhi_18.csv
loading vhi_6.csv
loading vhi_1.csv
loading vhi_2.csv
loading vhi_7.csv
loading vhi_5.csv


In [3]:
import pandas as pd
name='vhi_{}.csv'#название файла 
frame = pd.DataFrame() #фрейм - пустой список 
for i in range(1, 26): 
    temp = pd.read_csv(name.format(i), sep='[, ]+', engine='python')#читаем наш файл
    temp['Province'] = i
    frame = frame.append(temp, ignore_index=True)


df=frame

def getByProvinceAndYear(province, year): #функция для поиска данных по области и году
    return df[(df['Province'] == province) & (df['year'] == year)]['VHI']

recSeries = getByProvinceAndYear(4, 1994)#1 задание

def getYearsInRestrictions(province, minVHI, maxVHI, procent):
    years = df['year'].unique() #выбираем неповторяющиеся годы
    result = [] 
    for year in years:
        series = getByProvinceAndYear(province, year) #берем нужную нам облась и год
        count_in_range = series[(series >= minVHI) & (series <= maxVHI)].count() #считаем все, что нам подходит
        procent_in_range = count_in_range * 100 / series.count() #находим наш процент
        if procent_in_range >= procent: #если наш процент за год больше, чем нужно, то заполняем наш фрейм
            result.append(year)
    return result
print(recSeries.min())
print(recSeries.max())

import numpy as np

table = pd.read_csv('vhi_1.csv', sep='[, ]+', engine='python')
def getMean(province):
    year1 = 2007
    year2 = 2008
    weeks=province.week.unique()
    Mean1=[]
    Mean2=[]
    M=[]
    for y in weeks:
        df = province[(province.year == year1) & (province.week == y)]
        df1 = province[(province.year == year2) & (province.week == y)]
        Mean1.append(np.mean(df.VHI))
        Mean2.append(np.mean(df1.VHI))
        M.append(int(df.week//4.34 + 1))
    result = pd.DataFrame({'M':M,'weeks':weeks, 'mean1': Mean1, 'mean2': Mean2})
    return result

def getMean(df):
    year1 = 2007
    year2 = 2008    
    Mean1=[]
    Mean2=[]
    M=[]
    Province = [] 
    for province in df['Province'].unique():   
        province_df = df[df['Province'] == province]
        for i in range(1, 13):
            monthes = province_df[(province_df.week > 4*(i-1)) & (province_df.week <= 4*i)]
            month7 = monthes[(monthes.year == year1)]
            month8 = monthes[(monthes.year == year2)]
            Mean1.append(np.mean(month7.VHI))
            Mean2.append(np.mean(month8.VHI))
            M.append(i)
            Province.append(province)
    result = pd.DataFrame({'M':M, 'mean1': Mean1, 'mean2': Mean2, 'Province': Province})
    return result
        

22.5
82.05


In [4]:
getMean(frame)

,M,mean1,mean2,Province
0,1,48.9675,45.3225,1
1,2,46.5850,52.9150,1
2,3,46.1350,54.0700,1
3,4,46.1850,64.2925,1
4,5,42.7425,79.4350,1
...,...,...,...,...
295,8,14.7300,29.1100,25
296,9,14.3925,24.3200,25
297,10,25.2600,31.3075,25
298,11,39.3850,35.1975,25


In [5]:
def getMinMax(df):
    years = df['year'].unique()
    provinces = df['Province'].unique()
    Province = [] 
    Years = []   
    Min = []
    Max = []
    for province in provinces:
        for year in years:
            temp = df[(df['year'] == year) & (df['Province'] == province)]
            Min.append(temp['VHI'].min())
            Max.append(temp['VHI'].max())
            Province.append(province)   
            Years.append(year)
  
    result = pd.DataFrame({'Years':Years, 'min': Min, 'max': Max, 'Province': Province})
    return result

grouped = df.groupby(['Province', 'year'])
#print grouped['VHI'].agg([ np.min, np.max])
grouped.agg({'VHI':[np.mean]})



VHI
                    mean
Province year           
1        1982  40.285962
         1983  41.273846
         1984  53.313061
         1985  43.737955
         1986  34.906154
...                  ...
25       2015  43.349231
         2016  48.092115
         2017  40.737885
         2018  39.396346
         2019  36.242692

[950 rows x 1 columns]

In [76]:
getMinMax(frame)

,Years,min,max,Province
0,1982,19.92,59.42,1
1,1983,31.16,54.04,1
2,1984,31.54,76.06,1
3,1985,33.15,60.35,1
4,1986,23.15,44.45,1
...,...,...,...,...
945,2015,21.65,63.60,25
946,2016,31.14,67.79,25
947,2017,21.17,60.38,25
948,2018,22.83,50.09,25
